# Recursion

---

## Financial Team TWIL

> Shared by [laserx@github](https://github.com/laserx)

## References

* [Elixir offical guides (recursion)](https://elixir-lang.org/getting-started/recursion.html)
* [Weighted moving average](https://en.wikipedia.org/wiki/Moving_average#Weighted_moving_average)

---

## Loops through recursion

loops in Elixir (as in any functional programming language) are written differently from imperative languages.

for example, like golang:

```go
package main

import (
	"fmt"
)

func main() {
	array := []int{1, 2, 3, 4, 5}

	for i := 0; i < len(array); i++ {
		array[i] = array[i] * 2
	}

	fmt.Println(array)
}
```

In the example above, we are mutating both the array and the variable `i`.

Mutating is not possible in Elixir.

Instead, functional languages rely on recursion: a function is called recursively until a condition is reached that stops the recursive action from continuing.

In [1]:
defmodule Recursion do

  def run(input), do: resolve(input)

  defp resolve(input, output \\ [])
  defp resolve([], []), do: []
  defp resolve([h | t], output), do: resolve(t, output ++ [h * 2])
  defp resolve([], output), do: output
end

{:module, Recursion, <<70, 79, 82, 49, 0, 0, 5, 120, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 0, 140, 0, 0, 0, 16, 16, 69, 108, 105, 120, 105, 114, 46, 82, 101, 99, 117, 114, 115, 105, 111, 110, 8, 95, 95, 105, 110, 102, ...>>, {:resolve, 2}}

In [2]:
Recursion.run([1, 2, 3, 4, 5])

[2, 4, 6, 8, 10]

In [3]:
Recursion.run([])

[]

## Resolve EMA problem


An exponential moving average (EMA), also known as an exponentially weighted moving average (EWMA), is a first-order infinite impulse response filter that applies weighting factors which decrease exponentially. The weighting for each older datum decreases exponentially, never reaching zero. The graph at right shows an example of the weight decrease.

The EMA for a series Y may be calculated recursively:

\begin{align}S_t = \begin{cases}
 Y_1,                                           &t = 1 \\
 \alpha \cdot Y_t + (1 - \alpha) \cdot S_{t-1}, &t > 1
\end{cases}\end{align}

\begin{align}\alpha ={2 \over N+1}\end{align}



In [4]:
defmodule Indicatorex.EMA do
  @type t :: %Indicatorex.EMA{span: integer, v: [float]}
  defstruct span: 0, v: []

  @spec calc([number()], number()) :: {:error, String.t()} | {:ok, Indicatorex.EMA.t()}
  def calc(data, span) when span >= 1, do: run(data, span)
  def calc(_, span), do: {:error, "span value: #{span}. span must above 1"}

  @spec calc(number(), number(), number()) :: float()
  def calc(close, pre, span),
    do: (2 * close + (span - 1) * pre) / (span + 1)

  defp run(data, span, resp \\ [])
  defp run([], _span, []), do: {:error, "empty"}
  defp run([], span, resp), do: {:ok, %Indicatorex.EMA{span: span, v: resp}}
  defp run([h | t], span, []), do: run(t, span, [h / 1])

  defp run([h | t], span, resp) do
    [pre] = Enum.take(resp, -1)
    new = calc(h, pre, span)
    run(t, span, resp ++ [new])
  end
end

{:module, Indicatorex.EMA, <<70, 79, 82, 49, 0, 0, 12, 80, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 1, 19, 0, 0, 0, 33, 22, 69, 108, 105, 120, 105, 114, 46, 73, 110, 100, 105, 99, 97, 116, 111, 114, 101, 120, 46, 69, 77, 65, ...>>, {:run, 3}}

In [5]:
Enum.to_list(1..100) |> Indicatorex.EMA.calc(7)

{:ok, %Indicatorex.EMA{span: 7, v: [1.0, 1.25, 1.6875, 2.265625, 2.94921875, 3.7119140625, 4.533935546875, 5.40045166015625, 6.3003387451171875, 7.225254058837891, 8.168940544128418, 9.126705408096313, 10.095029056072235, 11.071271792054176, 12.053453844040632, 13.040090383030474, 14.030067787272856, 15.022550840454642, 16.01691313034098, 17.012684847755736, 18.009513635816802, 19.0071352268626, 20.00535142014695, 21.004013565110213, 22.00301017383266, 23.002257630374494, 24.001693222780872, 25.001269917085654, 26.00095243781424, 27.00071432836068, 28.000535746270508, 29.00040180970288, 30.00030135727716, 31.00022601795787, 32.00016951346841, 33.00012713510131, 34.00009535132598, 35.00007151349449, 36.000053635120864, 37.000040226340644, 38.00003016975548, 39.00002262731661, 40.00001697048746, 41.000012727865595, 42.000009545899196, 43.0000071594244, ...]}}